### LSE Data Analytics Online Career Accelerator

# DA201: Data Analytics using Python

## Assignment template: COVID-19 data

## Student Note
This template can be used to better understand the suggested assignment workflow and how to approach the questions. You are welcome to add code and Markdown blocks to the various sections to add either code or comments. Make sure to add code cells as applicable, and to comment all your code blocks.

You have the option to populate your Notebook with all the elements typically contained within the report, or to submit a separate report. In the case of submitting your Notebook, you can embed images, links and text where appropriate in addition to the text notes added.

**SPECIAL NOTE**
- Submit your Jupyter Notebook with the following naming convention: `LSE_DA201_assignment_[your name]_[your surname]` (remove the square brackets).
- You should submit a zipped folder containing all the elements used in your Notebook (data files, images, etc.).

> ***Markdown notes:*** Remember to change cell types to `Markdown`. You can review [Markdown basics](https://docs.github.com/en/get-started/writing-on-github/getting-started-with-writing-and-formatting-on-github/basic-writing-and-formatting-syntax) to find out how to add formatted text, links and images to your Markdown documents.

## 0) Environment preparation
These settings are provided for you. You do not need to make any changes.

In [ ]:
# Import the required libraries and set the plotting options
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(rc = {'figure.figsize':(15,10)})

## 1) Assignment activity 1: 

### 1.1) Report/notebook expectations:
- Illustrate your GitHub setup consisting of the load and push updates of all the Jupyter Notebook files. (**Hint**: Make sure that your GitHub username, the directory structure and updates are visible in the screenshot. Make sure to provide a zipped copy of the folder containing your submission Notebook as well as supporting files such as images used in the Notebook.)

### Required: Report submission:
Insert URL (to your public GitHub repository) and a screenshot - double click cell to edit
- [My Github Repo](https://github.com/username/reponame)
- Screenshot demo (replace with your own).

!['My Github screenshot](http://github.com/apvoges/lse-ca/blob/main/GitHubScreenshot.png?raw=true)
(Note that this only works if your repo is set to **public**. Alternatively you need to refer to a local image and include this image in your submission.)

### 1.2) Presentation expectations:
- Describe the role and how workflow tools such as GitHub can be used to add value to organisations.

### Optional for notebook/Required for presentation.
- You can use this cell as placeholder for bullet points to include in your presentation. 
- This section will not be graded in the Notebook, grades awarded based on presentation content only. 

(Double-click to edit)

## 2) Assignment activity 2: 

### 2.1) Report expectations:
- Load the files `covid_19_uk_cases.csv` and `covid_19_uk_vaccinated.csv` and explore the data.
- Explore the data using the `info()`, `describe()`, `shape` and `value_counts()` methods, and note the observations regarding data types, number or records and features.
- Identify missing data.
- Filter/subset data.
- Aggregate data (totals and by month).
- Note observations.

In [ ]:
# Load the COVID-19 cases and vaccine data sets as cov and vac respectively

In [ ]:
cov = pd.read_csv('covid_19_uk_cases.csv')
vac = pd.read_csv('covid_19_uk_vaccinated.csv')

In [ ]:
# Explore the DataFrames with the appropriate functions
# sense check the DataFrames
cov

In [ ]:
vac

## Sense check the data
To get a better idea of the data, I opened the csvs in excel so that I would be able to check if the correct number of rows and columns imported for each data set. I used the shape and columns functions to see this.

In [ ]:
# View the number of rows and columns of the DataFrames to sense check the data.
print(cov.shape)
print(vac.shape)

In [ ]:
# View the column names included in each data set.
print(cov.columns)
print(vac.columns)

## Initial exploration of the data
In this step, I determine the data types of each column in each data set and identify the number of missing values in the data sets. I still need to decide how to treat the missing values and if the data types of each column are acceptable for analysis. 

In [ ]:
# Determine the data types of each column in the cov data sets. 
print(cov.dtypes)

In [ ]:
# Determine the data types of each column in the cov data sets.
print(vac.dtypes)

#### Objects are categorical variables.
This implies that every object which is not an integer or float is a categorical variable. The only column for which this might need to be changed is the 'Date' column. Below, I view the date columns to get a sense of whether the data was recorded continuously over time or on distinct reporting days.

In [ ]:
# view the date column
cov['Date']

Dates are recorded continuously with a period of one day.I suspect that the data is classified as categorical because for each region, there will be an entry for that day. The number of covid cases on January 1st 2020 for each region can be compared. In order to study this data overtime, dates must be converted to datetime data types. 

In [ ]:
# Convert the date column to the correct object
cov['Date'] = pd.to_datetime(cov['Date'])
vac['Date'] = pd.to_datetime(vac['Date'])

# Sense check the data
print("Date type for vac data: ", vac.dtypes['Date'])
print("Date type for vac data: ", cov.dtypes['Date'])

In [ ]:
# Determine the number of missing values in the cov data set.
cov.isna().sum()

In [ ]:
# Determine the number of missing values in the vac data set.
vac.isna().sum()

In [ ]:
# View the rows containing missing values
cov[cov.isnull().any(axis=1)]

In [ ]:
# Forward fill the missing values
cov = cov.fillna(method='ffill')

# Sense check to ensure the missing values have been imputed
cov[cov.isnull().any(axis=1)]

In [ ]:
# Show the previously missing values
cov.iloc[875:877]

It can be seen that only the cov data set has missing values. These are in the 'deaths', 'cases', 'recovered' and 'hospitalised' columns. 

In [ ]:
# Set the index of the datasets to 'Date to enable time series analysis'
cov = cov.set_index('Date')
vac = vac.set_index('Date')

# Explore the Distribution of the data as a whole

The data is first normalised, to enable comparison between datasets at a later stage. I will use maximum absolute scaling to determine to scale data points between values of -1 and 1. I hope to identify if the distributions of the two subsets differ. 

In [ ]:
# Define maximum absolute scaling as a function
def max_abs_scaling(df): #define a function that uses a df as input
    #copy the DataFrame
    df_scaled = df.copy()
    if isinstance(df_scaled, pd.DataFrame):    #checks if the df given is a df
        #apply maximum absolute scaling
        for column in df_scaled.columns:   #divide the value by the abs(max(value of the col))
            df_scaled[column] = df_scaled[column] / df_scaled[column].abs().max()
            
    else:
        df_scaled = df_scaled / df_scaled.abs().max()   #if not a dataframe, still norm. 
    
    return df_scaled

In [ ]:
# View the distribution of the data as a whole

# Keep only the relevant columns for comparison
cases_study = cov[['Cases','Hospitalised', 'Recovered', 'Deaths']]

# Normalise the data
normal_cases = max_abs_scaling(cases_study)

# View the distribution of the whole data
sns.boxplot(data=normal_cases).set(title='The Distribution of Covid Data (normalised)')

In [ ]:
# Group the data appropriately
cov_1 = cov.groupby(['Province/State'])[['Cases','Recovered', 'Hospitalised', 'Deaths']].sum()

# Normalise the data to make comparable
cov_1 = max_abs_scaling(cov_1)

cov_1 = cov_1.reset_index()

# Create a pairplot to identify outliers in the Covid data.
sns.pairplot(data=cov_1, hue = 'Province/State')

plt.ticklabel_format(style='Plain')

'Others' is an outlying value in every category. It should there be removed from the Covid data.

In [ ]:
# Group the data appropriately
vac_1 = vac.groupby(['Province/State'])[['Vaccinated','First Dose','Second Dose']].sum()

# Normalise the data to make comparable
vac_1 = max_abs_scaling(vac_1)

vac_1 = vac_1.reset_index()

# Create a pairplot to identify outliers in the Covid data.
sns.pairplot(data=vac_1, hue = 'Province/State')

plt.ticklabel_format(style='Plain')

'Others' does not seem to be have outlying values in the vaccine data, but Gibraltar sits as the maximum value on each graph. Since the data has been normalised and aggregated, that implies that Gibraltar had the highest number of partially and fully vaccinated people per province. Gibraltar will be explored further to understand if this is accurate or erroneous.

'Others' will be removed from the DataFrame that combines both the vaccine data and the covid data.

## Explore the Gibraltar data
The below code can be used to subset and explore each region included in the data. 

In [ ]:
# View the provinces included in the data sets 
print(cov['Province/State'].unique())

print(vac['Province/State'].unique())

It is expected that each province would have similar distributions of cases. To identify data which is could be erroneous, the distributions of each province are displayed using boxplots. 

In [ ]:
# Create DataFrame based on Gibraltar data
# Hint: newdf = df[df[col]==index]
gibraltar_cases = cov[cov['Province/State']=='Gibraltar'][['Cases','Hospitalised', 'Recovered', 'Deaths']]

gibraltar_vac = vac[vac['Province/State']=='Gibraltar'][['Vaccinated', 'First Dose', 'Second Dose']]

# print the whole DataFrame
pd.set_option("display.max_rows", None)

# view the DataFrames to sense check
print(gibraltar_cases.shape)
print(gibraltar_vac.shape)

In [ ]:
# Explore behaviour over time

In [ ]:
# compare Gibraltar data to Anguilla, and to the data as a whole 

anguilla_cases = cov[cov['Province/State']=='Anguilla'][['Cases','Hospitalised', 'Recovered', 'Deaths']]

anguilla_vac = vac[vac['Province/State']=='Anguilla'][['Vaccinated', 'First Dose', 'Second Dose']]

# print the whole DataFrame
pd.set_option("display.max_rows", None)

# view the shape of the DataFrames to sense check
print(anguilla_cases.shape)
print(anguilla_vac.shape)

Comparing the two subsets is not entirely useful as their population sizes are different. I have chosen to normalise the Anguilla and the Gibraltar subsets to understand if their distributions are similar. I will use maximum absolute scaling to determine to scale data points between values of -1 and 1. I hope to identify if the distributions of the two subsets differ.  

In [ ]:
# apply maximum absolute feature scaling to the Anguilla data
normal_anguilla_cases = max_abs_scaling(anguilla_cases)

# view descriptive statistics for normalised data
normal_anguilla_cases.describe()

In [ ]:
# do the same for Gibraltar
normal_gibraltar_cases = max_abs_scaling(gibraltar_cases)

# view descriptive statistics for normalised data
normal_gibraltar_cases.describe()

In [ ]:
# view the distribution of the Gibraltar data
sns.boxplot(data=normal_gibraltar_cases).set(title = 'The distribution of Gibraltar data (normalised)')

In [ ]:
# View the distribution of the Anguilla data
sns.boxplot(data=normal_anguilla_cases).set(title = 'The distribution of Anguilla data (normalised)')

Gibraltar's range is much higher than Anguilla's. This indicates that there could be a problem with how Gibraltar's data was collected. Since this data is normalised, I would expect each province to have similar ranges. Gibraltar's data suggests that it has much higher numbers, in categories which differ from Anguilla's (for example, the Deaths and Hospitalisation columns). Further, Gibraltar's maximum number of deaths exceeds it's maximum number of cases. This would mean that more people died from Covid-19 in Gibraltar than the highest number of daily cases recorded. This is unreasonable and implies that something is wrong with the Gibraltar data. 

Anguilla mirrors the distribution of the data as a whole. There are many outliers in the whole data set, as well as in Anguilla's distribution. The range of Gibraltar's values in each category is much higher than both the total dataset and the Anguilla. I will now remove Gibraltar from the dataset and check the distribution again, to see if the data has less outlying high values. 

In [ ]:
# Create a dataframe without Gibraltar. 
cases_clean = cov[cov['Province/State'] != 'Gibraltar'][['Province/State','Cases','Hospitalised', 'Recovered', 'Deaths']].copy()

# Keep only the relevant columns for comparison
cases_clean_study = cases_clean[['Cases','Hospitalised', 'Recovered', 'Deaths']]

#normalise
cases_clean_normal = max_abs_scaling(cases_clean_study)

# View the distribution of the whole data
sns.boxplot(data=cases_clean_normal)

The data's distribution stays fairly constant after removing the Gibraltar dataset. 

However, since there were errors in the collection of Gibraltar's Covid dataset, Gibraltar is removed from both the vaccine and Covid datasets to keep the number of provinces studied consistent. 

In [ ]:
# Remove Gibraltar from vaccine and the covid datasets. 
cov_clean = cov[cov['Province/State']!='Gibraltar']
vac_clean = vac[vac['Province/State']!='Gibraltar']

In [ ]:
# sense check Covid data
cov_clean.loc[cov_clean['Province/State'] == 'Gibraltar']

In [ ]:
# sense check vaccine data
vac_clean.loc[vac_clean['Province/State'] == 'Gibraltar']

### 2.2) Presentation expectations:
Use the process of exploring the data for Gibraltar as an example to provide a brief description of the various phases to help your team to understand the process. Keep it high level and make sure to focus on both specifics relating to the case (first dose, second dose per region, total and over time) and brief observations regarding the process. Assignment activity 2 considers basic data exploration.
- Can we make decisions based on total numbers only, or do trends over time offer additional insights?
- Why it is important to explore the data, what are the typical mistakes made in this phase?

## 3) Assignment activity 3: 

### 3.1) Report expectations:
- Merge and explore the data.
- Convert the data type of the Date column from object to DateTime.
- Create a data set that meets the expected parameters.
- Add calculated features to DataFrames (difference between first and second dose vaccinations).
- Filter and sort output.
- Observe totals and percentages as a total and over time.
- Note observations.

Merge the DataFrames without duplicating columns. The new DataFrame (e.g. `covid`) will have `7584` rows and the following columns: `Province/State, Country/Region, Date, Vaccinated, First Dose, Second Dose, Deaths, Cases, Recovered, Hospitalised`.

## The type of join used
An inner join contains overlapping data in each dataset based on the join columns. The resulting dataframe must contain the correct data for the date and Province, and considering the data matches date for date, an inner join is appropriate. 

In [ ]:
# Join the DataFrames as covid where you merge cov and vac
covid = pd.merge(cov_clean, vac_clean, on=['Province/State','Date'], how='inner')

#covid.head()
print(covid.columns)
print(covid.loc[covid['Province/State'] == 'Gibraltar'])

covid.head()

In [ ]:
# Remove 'Others' from the covid DataFrame
covid = covid[covid['Province/State'] != 'Others']

In [ ]:
# Explore the new DataFrame
print(covid.shape)

# identify the column names and column data types
print(covid.columns)
print(covid.dtypes)

The columns Country/Region, Lat, Long, ISO 3166-1 Alpha 3-Codes, Sub-region Name and Intermediate Region Code are duplicated and can be dropped from the covid DataFrame. 

In [ ]:
# Clean up / drop unnecessary columns 
dict1 = {'Province/State': 'Province/State', 'Country/Region_x':'Country/Region', 'Lat_x':'Lat',
         'Long_y':'Long','ISO 3166-1 Alpha 3-Codes_x' : 'ISO 3166-1 Alpha 3-Codes',
         'Sub-region Name_x':'Sub-region Name', 
         'Intermediate Region Code_x':'Intermediate Region Code', 'Date':'Date', 
        'Cases':'Cases', 'Recovered':'Recovered', 'Hospitalised':'Hospitalised', 
         'Vaccinated':'Vaccinated', 'First Dose':'First Dose', 'Second Dose': 'Second Dose'}

covid.rename(columns=dict1, inplace=True)

print(covid.columns)
covid.head()

In [ ]:
# Groupby and calculate difference between first and second dose
covid['Difference_Doses'] = covid['First Dose'] - covid['Second Dose']

covid.groupby('Province/State')[['Difference_Doses']].sum().sort_values('Difference_Doses')

### Observations
All provinces have a positive difference between first and second doses. This means that more people across provinces had the first dose than the second dose. We see that Montserrat had the biggest difference between people who had the first dose and those who had the second. Saint Helena had the lowest difference. 

In [ ]:
# Reset the index to groupby month
covid = covid.reset_index()

covid['Month']=covid['Date'].dt.to_period('M')
vac_monthly = covid.groupby(['Province/State','Month'])[['Difference_Doses']].sum()
vac_monthly

It can be seen that for across the provinces, the monthly difference between those who have had their first dose and those who have had their second dose becomes negative in April and continues to be negative until October. This shows that after March, more people got their second dose than their first. 
The difference is at its lowest value in April and slowly increases towards October.  
In October of 2021, more people got their first dose than their second dose . 
This implies that in April, a lot of people were getting their second dose. Slowly more people started to get their first dose while less received their second.

### 3.2) Presentation expectations:
We use similar calculations and representations as we had in assignment activity 2, but now expand to look at all provinces. Assignment activity 3 is concerned with exploring data in the context of a specific business question (as opposed to general exploration in assignment activity 2).
- What insights can be gained from the data? (Description of all regions, assumptions and concerns, trends or patterns you have observed.)
- Are there limitations or assumptions that needs to be considered?
- Make sure to provide a brief overview of the data and typical considerations at this phase of analysis.

In [ ]:
# Which province has the highest number of partially vaccinated individuals? 
vaxed_totals = covid.groupby(['Province/State', 'Month'])[['Vaccinated','First Dose', 'Second Dose', 'Difference_Doses']].sum()

print(vaxed_totals.loc[vaxed_totals['First Dose'].idxmax()])
print()
print(vaxed_totals.loc[vaxed_totals['First Dose'].idxmin()])

In [ ]:
# Which province has the highest percentage of partially vaccinated individuals overall? 

# Calculate the monthly percentage of vaccinated individuals 
vaxed_totals['Partially Vaccinated'] = (vaxed_totals['First Dose'] / (vaxed_totals['First Dose'] + vaxed_totals['Vaccinated']) ) * 100

vaxed_totals.dropna()

In [ ]:
# Find the city with the highest % of partially vaccinated people
vaxed_totals.loc[pd.IndexSlice[:, '2021-09', :]].idxmax()

## 4) Assignment activity 4: 

The government is looking to promote second dose vaccinations and would like to know the best possible area to test a new campaign. They are looking for the highest number of people who have received a first dose and not a second dose. 
- Where should they target?
- Which provinces have the highest number (actual numbers) and highest relative numbers (second dose only/first dose)?
- Visualise both outputs.

### 4.1) Report expectations:
- Consider additional features (deaths and recoveries).
- Visualise the data.
- Note observations:
 - Do deaths follow the same patterns observed in vaccination data (daily vs cumulative)?
 - Do we need to separate groups of data for specific variables and analyse them in isolation (Others) to be able to observe the patterns?

### 4.2) Presentation expectations:
- What insights can be gained from the data?
- Why do we need to consider other features?
- **Hints**: 
 - Evalute different features to improve decision making (deaths and recoveries). 
 - Why it is important to explore data and use different views?
 - Highlight two or three suggestions to get junior team members started in terms of good practices.

In [ ]:
# Absolute numbers

In [ ]:
# Calculate difference between first and second dose
covid['Difference Doses'] = covid['First Dose'] - covid['Second Dose']

# Calculate the total difference by province
doses_by_province = covid.groupby('Province/State')[
                    ['First Dose', 'Second Dose','Difference Doses']].sum().sort_values('Difference Doses')

# View the data
doses_by_province

In [ ]:
# Calculate the ratio of interest
doses_by_province['Eligible (%)'] = (doses_by_province['Difference Doses'] / 
                                 doses_by_province['First Dose']) * 100

# Calculate the ratio of people who have received first dose to second
doses_by_province['First : Second'] = (doses_by_province['First Dose'] / 
                                 doses_by_province['Second Dose'])

doses_by_province = doses_by_province.reset_index()
print(doses_by_province.dtypes)

In [ ]:
fig, ax = plt.subplots()
ax.ticklabel_format(style='plain')

sns.barplot(x = 'First : Second', y='Province/State', data=doses_by_province)


All recorded provinces have the same ratio of people who have received their first dose to those that have received their second. A better approach would be to see if the number of people going to get their second dose is increasing or decreasing over time. 

In [ ]:
# Visualise the ratio of total first doses to total second doses
doses_by_province[['First Dose', 'Second Dose']].plot(kind='bar', stacked=True).set_xticklabels(doses_by_province['Province/State'])
# ax.ticklabel_format(style='plain')
plt.title("Comparison between First Dose and Second Dose by Province")

Here we can see that the ratio of aggregated fully vaccinated individuals is roughly equal to the number of people who received their first dose. Once again, a better indicator would be to consider the trend in vaccination. 

In [ ]:
# Communicate the number of deaths per month per region
fatalities = covid.groupby(['Province/State', 'Month'])[['Deaths']].sum()
fatalities = fatalities.reset_index(['Province/State','Month'])

# Reset the Month column data type to DateTime
fatalities['Month']=pd.PeriodIndex(fatalities['Month'], freq='M').to_timestamp()

# Sense check the data types
print(fatalities.dtypes)

fatalities

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.lineplot(x='Month', y='Deaths', hue='Province/State', data=fatalities)

We see that in all provinces, the deaths per month are decreasing. The straight line part of each graph indicates that the  numbers of deaths are decreasing at a constant rate. This indicates that a peak in the daily deaths has been reached across most provinces. 
The Channel Islands has the highest number of monthly deaths out of all provinces. From our above vaccine data, they have an average rate of vaccination. Monteserrat has the highest number of partially and fully vaccinated people and one of the lowest deaths per month lines. 

In [ ]:
# Communicate the number of recoveries per region
recoveries = covid.groupby(['Province/State'])[['Recovered']].sum()
recoveries = recoveries.reset_index()
recoveries

In [ ]:
# Visualise recoveries in total
fig, ax = plt.subplots()
ax.ticklabel_format(style='plain')
#ax.set_xticklabels( 'Province/State', rotation=90)

sns.barplot(x = 'Recovered', y='Province/State', data=recoveries)

plt.show()

Observations:
The Channel Islands had the largest number of recoveries. 

In [ ]:
# Is this consistent with the number of cases recorded? 
cases = covid.groupby(['Province/State'])[['Cases','Recovered', 'Hospitalised', 'Deaths']].sum()
cases = cases.reset_index()
sns.pairplot(data=cases, hue = 'Province/State', x_vars=['Cases', 'Deaths', 'Hospitalised'],
            y_vars=['Recovered'])
plt.ticklabel_format(style='Plain')

This pairplot shows that the Channel Islands has the highest total deaths, hostpitalisations, cases and recoveries out of all of the Provinces. Most noticeably, the Channel Islands had the largest number of deaths for quite a low number of hospitalizations. The Channel Islands was not shown to be an outlier in my initial exploration and cleaning. I will consider the data as not erroneous, but rather note that the Channel Islands may be a good target for the vaccination campaign. 

In [ ]:
# Is this consistent overtime? 
recoveries_month = covid.groupby(['Province/State', 'Month'])[['Recovered']].sum()
recoveries_month = recoveries_month.reset_index(['Province/State','Month'])

# Reset the Month column data type to DateTime
recoveries_month['Month']=pd.PeriodIndex(recoveries_month['Month'], freq='M').to_timestamp()

In [ ]:
# Visualise recoveries overtime
sns.set(rc = {'figure.figsize':(15,8)})
sns.lineplot(x='Month', y='Recovered', hue='Province/State', data=recoveries_month)

The Channel Islands has consistently had the highest recovery rate over the period of the data. From the previous graph, it is known that the Channel Islands was an outlying Province. It's data does follow the same trends as the other Provinces. There is a clear peak in July of 2021, and after that a steep decline in recoveries. This coincides with an increase in cases starting a bit before July 2021 (case graph seen below). The Channel Islands remains a good target for the marketing campaign.  

In [ ]:
# Other features evaluated (data preparation, output and plots)
# Rate of Second Dose vaccination over time

In [ ]:
doses_overtime = covid.groupby(['Province/State', 'Month'])[['First Dose', 'Second Dose' , 'Vaccinated']].sum()
doses_overtime = doses_overtime.reset_index(['Province/State','Month'])

# Reset the Month column data type to DateTime
doses_overtime['Month']=pd.PeriodIndex(recoveries_month['Month'], freq='M').to_timestamp()

doses_overtime

In [ ]:
# Visualise second doses overtime overtime
sns.set(rc = {'figure.figsize':(15,8)})

fig, ax = plt.subplots()
ax.ticklabel_format(style='plain')

sns.lineplot(x='Month', y = 'Second Dose', hue='Province/State', data=doses_overtime).set(title='Second Dose Overtime')

plt.show()

In [ ]:
# visualise first dose overtime
fig, ax = plt.subplots()
ax.ticklabel_format(style='plain')

g = sns.lineplot(x='Month', y='First Dose', hue='Province/State', data=doses_overtime)
g.set(title='First Doses Overtime')
plt.show()

Across all provinces, there is a trend in the uptake of vaccinations. Between January and April of 2021, the number of people getting their first dose starts to plateau and then begins to decrease. The number of people getting their second dose starts to increase at the point where first doses start to decrease. This makes logical sense, as more people who had their first dose will be able to move onto getting their dose. 

The second decrease in first dose vaccinations comes before September of 2021. Second doses are starting to plateau during this time as well. After September, the rate at which first doses decrease across provinces starts to decrease as well. This indicates thats people are slowly starting to need their first dose again. According to the case graph, cases are still decreasing. 

People may be leaning towards getting their first dose, but with cases declining the sentiment around getting vaccinated may turn more negative or apathetic. 

In [ ]:
# has there been a peak in cases reported? 
# Is this consistent overtime? 
cases_overtime = covid.groupby(['Province/State', 'Month'])[['Cases']].sum()
cases_overtime = cases_overtime.reset_index(['Province/State','Month'])

# Reset the Month column data type to DateTime
cases_overtime['Month']=pd.PeriodIndex(cases_overtime['Month'], freq='M').to_timestamp()

# View the data
cases_overtime

In [ ]:
# Visualise case data
fig, ax = plt.subplots()
ax.ticklabel_format(style='plain')

sns.lineplot(x='Month', y = 'Cases', hue='Province/State', data=cases_overtime)

plt.show()

***Notes and observations:***
Your observations here. (Double click to edit)

***Examples could include:***
- Are there other trends in terms of recoveries or hospitalisations compared to other features that you found interesting and that may add value in terms of the decision making process?
- Any other observations regarding the data?
- Any suggestions for improvements and further analysis?
- What would your future data requirements be?

## 5) Assignment activity 5: External data 
In the next section, you were supplied with a sample file and the question was asked to determine whether there are additional `#tags` or keywords that could potentially provide insights into your COVID-19 analysis. While the sample set is limited, you were asked to review the provided file and demonstrate the typical steps and make recommendations regarding future use of similar data sets to provide richer insights.

### 5.1) Report expectations:
- Demonstrate basic ability to work with Twitter data.
- Search for hash-tags or keywords.
- Create DataFrames and visualisations.
- Note your observations.

In [ ]:
# Import the tweet data set
import pandas as pd
tweets = pd.read_csv('tweets.csv')

In [ ]:
# Explore the data: info(), head()
print(tweets.info())
print()

print(tweets.shape)

In [ ]:
# Explore the structure, count the tweets, get the elements of interest

# Identify rows with null value
tweets_null = tweets[tweets.isna().any(axis=1)]
tweets_null

### Look for Hashtags

In [ ]:
# Create a DataFrame with the text only
text = []
rows = tweets.shape[0]
#print(rows)
for count in range(0, rows):
    
    result = tweets['text'].loc[count]
    text.append(result)
    count += 1

print(type(text))
tweets_text = pd.DataFrame(text)
tweets_text

# check for null values
tweets_text.isnull().sum()

In [ ]:
# drop null values
tweets_text = tweets_text.dropna()
tweets_text

In [ ]:
# Norah's way
tweets['text'] = tweets['text'].astype(str)
tweets_text = tweets['text'].apply(lambda x: x if x.strip() != None else None)
tweets_text.head(15)

In [ ]:
# Find the hashtags
tags = []

for y in [x.split(' ') for x in tweets_text.values]:
    for z in y:
        if '#' in z:
            tags.append(z)
            
            
# Create a series containing a count of our values
tags = pd.Series(tags).value_counts()
tags.head(30)

In [ ]:
# Convert to a DataFrame
tweets_tags = pd.DataFrame(tags)
tweets_tags.head(30)

In [ ]:
# Reset the index and rename the columns
tweets_tags = tags.reset_index()

# Rename the columns
tweets_tags.columns = ['Hashtag', 'Count']

# Sense check the data
tweets_tags.head(15)

In [ ]:
# Check data types
print(tweets_tags.dtypes)

In [ ]:
# Display hashtags that appeared more than 100 times
display(tweets_tags.loc[(tweets_tags['Count'] > 100)])

In [ ]:
# Visualise the hashtag counts
import seaborn as sns
sns.barplot(x='Count', y='Hashtag', data=tweets_tags.loc[(tweets_tags['Count'] > 100)]).set(title='Popular Hashtags')

### Look for common keywords associated with the data

In [ ]:
tweets_text.head(30)

In [ ]:
# develop a common words library
common = ['the', 'and', 'it', 'by', 'new', 'my', 'read', "we're", 'need', 'us', 'they',
         "they're", 'me', 'people', 'with', 'those', 'told', 'said', 'links', 'to', 
         'go', 'going', 'is', 'not', ' which', 'many', "can't", 'on', 'behind', 'and', 
         'much', 'but', 'a', 'lot', 'where', 'you', 'of', 'in', 'for', 'from', 'are', 'that'
          '&amp', 'The', '-', 'at', 'I', 'have', 'as', 'this', 'be', 'has', 'was', 'more', 'all', 'or',
          'we', 'will', 'can', 'an', 'been', 'about', 'our', 'your', 'now', 'their', 'over', 'after', 
          'cause', 'get', 'still', 'just', 'please', 'Please', 'than', 'a', 'do', 'since', 
          'into', 'what', '@', 'there','https', 'any', 'It', '\n', 'he', 'her', 'say', 'As', 
          "it's", 'want', 'keep', " ", '&amp;','that', 'who', 'its', 'being', 'up', 'like', '|', 'up', 'like', "I'm", 'how', 'cause', 
          'get', 'via', 'first', 'had', 'no', 'may', 'way', 'were','out', 'per', "I'm", 'safe', 'so',
         '-', 'during', 'A', 'For', 'if', 'one']

common_words = pd.Series(common)

In [ ]:
# Convert the tweets DataFrame into a list of tweets
tweets_text_values = tweets_text.values
tweets_text_values.tolist()

In [ ]:
# Strip out the words in the text which are not in the library above

# Create a list of words
t = [y.split(' ') for y in tweets_text_values]

# Flatten the list
import re
tweets_words_split = []
for i in t:
    for j in i: 
        if j.find(',') != -1:
            j = re.sub(",","", j)
            tweets_words_split.append(j)
        else:
            tweets_words_split.append(j)

# Remove https and common words
words = []
for x in tweets_words_split:
    if x not in common: 
        if x.find('https') != -1 or not x.startswith('#') :
            words.append(x)
            
# Create a series containing the counts of common words
words_S = pd.Series(words).value_counts()
words_S

In [ ]:
# Create DataFrame
words_df = pd.DataFrame(words_S)

words_df

In [ ]:
# reset the words index
words_df = words_df.reset_index()
words_df

In [ ]:
# Reset the column name for ease
words_df.columns = ['Word', 'Count']
words_df

In [ ]:
# Choose words with 200 or more uses
display(words_df.loc[(words_df['Count'] > 90)])

In [ ]:
# Display the common key words
sns.barplot(x='Count', y='Word', data=words_df.loc[(words_df['Count'] > 90)]).set(title='Popular Common Words')

I chose to look at words with 90 or more mentions. It it evident that something happened in Greece at the time of the data collection. 

Other interesting common word associations were Boris Johnson, 'protection', 'deaths'and 'signs'. 

While this does not say too much about what people's sentiment towards the Covid Vaccine is, it can be noted that people are talking about the vaccine and Covid-19. The government can use this to their advantage and promote their vaccination strategy online. 

Using words such as deaths or Deaths, cases and vaccine would be able to build traction online when the campaign is actually implemented. 

The best hashtags to use are "Covid19" and "CovidIsNotOver". This corresponds perfectly with what the recoveries, hospitalizations and deaths data indicates: that a peak in cases is over. People may be becoming more relaxed with restictions and cases may begin to rise again. 

### 5.2) Presentation expectations:
Discuss whether external data could potentially be used and whether it is a viable solution to pursue. Discuss your assumptions and suggestions. 

Points to consider:
- What insights can be gained from the data?
- What are the advantages and disadvantages of using external data?
- How would you suggest using external data in the project?

External data, when scraped from the internet, is cheap to aquire. It enables data from different sources to be used in one project. This enables different opinions or view point to be included in the analysis which may not have been included using data provided from the client. 

External data must be used ethically, and there can be gaps between what is legal and what is ethical. It may also be erroneous and introduce incorrect data into the analysis. Data scraped from websites like Twitter is not owned by the analyst but licensed by the company. This limits how you can use the data. 

Scraped data also has to be updated frequently to ensure it is accurate. 

In this project, I would use the data to try understand what people are talking about in association to Covid. This can give insight into what hashtags to use and what "buzz words" will inform the campaign marketing materials.

## 6) Assignment activity 6: 

### 6.1) Report expectations:
- Demonstrate using external function and interpret results.
- Note observations.

In [ ]:
# You can copy and paste the relevant code cells from the provided template here.

In [ ]:
# Load and select relevant subset of the data
# Make sure to change the relative path to function in your environment
ds1 = pd.read_csv('covid_19_uk_cases.csv')
ds2 = pd.read_csv('covid_19_uk_vaccinated.csv')

sample = ds1[['Province/State','Date','Hospitalised']]

# Sense check the data
print(sample.head)
print(sample.shape)

In [ ]:
# Select data for the Ilse of Man
sample_ci = sample[sample['Province/State'] == "Isle of Man"]

# Sense check the data
print(sample_ci.head())
print(sample_ci.shape)

In [ ]:
# Define function to plot moving averages
def plot_moving_average(series, window, plot_intervals=False, scale=1.96):
    
    # Create a rolling window to calculate the rolling mean using the series.rolling function
    rolling_mean = series.rolling(window=window).mean()
    
    # Declare the dimensions for the plot, plot name and plot the data consisting of the rolling mean from above 
    plt.figure(figsize=(18,4))
    plt.title('Moving average\n window size = {}'.format(window))
    plt.plot(rolling_mean, 'g', label='Simple moving average trend')

    
    # Plot confidence intervals for smoothed values
    if plot_intervals:
        
        # Calculate the mean absolute square 
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        
        # Calculate the standard deviation using numpy's std function
        deviation = np.std(series[window:] - rolling_mean[window:])
        
        # Calculate the upper and lower bounds 
        lower_bound = rolling_mean - (mae + scale * deviation)
        upper_bound = rolling_mean + (mae + scale * deviation)
        
        # Name and style upper and lower bound lines and labels 
        plt.plot(upper_bound, 'r--', label='Upper bound / Lower bound')
        plt.plot(lower_bound, 'r--')
    
    # Plot the actual values for the entire timeframe
    plt.plot(series[window:], label='Actual values')
    plt.legend(loc='best')
    plt.grid(True)

In [ ]:
# Define function to calculate the mean absolute error
def mean_absolute_error(a, b): return abs(b - a)

In [ ]:
# Demonstrate the use of the function to plot moving averages

# Convert the dates in the sample_ci data to datetime 
sample_ci['Date'] = pd.to_datetime(sample_ci['Date'])
print(sample_ci.dtypes)

# Set the date as the index
sample_ci = sample_ci.set_index('Date')

# Plot the moving averages
plot_moving_average(sample_ci['Hospitalised'], 7, plot_intervals=True)

In [ ]:
# Return the top three days with biggest difference between daily value and rolling 7-day mean
s = sample_ci.copy()
s_rolling = s['Hospitalised'].rolling(window=7).mean()
s['error'] = mean_absolute_error(s['Hospitalised'][7:], s_rolling[7:])
print(s.sort_values('error', ascending=False).head(3))

Mean absolute error represents the distance between the rolling 7 day average for a data point and the true value of the data point itself.  It is often used as a measure of the accuracy of the model. 

The above code block shows the largest differences between the moving average estimate of a data point and the true value. This is misleading because Mean Absolute Error is actually calculated by aggregating the absolute error terms to give an indication of how well a model performs on average.

In [ ]:
# Calculate the average of the absolute error terms. 
s['error'].mean()

In [ ]:
# Repeat for Turks and Caicos
sample_ci2 = sample[sample['Province/State'] == "Turks and Caicos Islands"]

# Convert the dates in the sample_ci data to datetime 
sample_ci2['Date'] = pd.to_datetime(sample_ci2['Date'])
print(sample_ci2.dtypes)

# Set the date as the index
sample_ci2 = sample_ci2.set_index('Date')

# Sense check the data
print(sample_ci2.head())
print(sample_ci2.shape)

In [ ]:
# Plot the moving averages
plot_moving_average(sample_ci2['Hospitalised'], 7, plot_intervals=True)

In [ ]:
# Repeat for British Virgin Islands
sample_ci3 = sample[sample['Province/State'] == "British Virgin Islands"]

# Convert the dates in the sample_ci data to datetime 
sample_ci3['Date'] = pd.to_datetime(sample_ci3['Date'])
print(sample_ci3.dtypes)

# Set the date as the index
sample_ci3 = sample_ci3.set_index('Date')

# Sense check the data
print(sample_ci3.head())
print(sample_ci3.shape)

In [ ]:
# Plot the moving averages
plot_moving_average(sample_ci2['Hospitalised'], 7, plot_intervals=True)

The above places of interest have been indentified as suitable targets for the government's campaign. Below the Channel Islands are checked as they were treated as outlying data during the analysis. 

In [ ]:
# Place of interest: Channel Islands
sample_ci4 = sample[sample['Province/State'] == "Channel Islands"]

# Convert the dates in the sample_ci data to datetime 
sample_ci4['Date'] = pd.to_datetime(sample_ci4['Date'])
print(sample_ci4.dtypes)

# Set the date as the index
sample_ci4 = sample_ci4.set_index('Date')

# Sense check the data
print(sample_ci4.head())
print(sample_ci4.shape)

In [ ]:
# Plot the moving averages
plot_moving_average(sample_ci2['Hospitalised'], 7, plot_intervals=True)

The Channel Islands follow the same pattern as the other three Provinces. I conclude that hospitalisations reached a peak after January of 2021, ending during February.

### 6.2) Presentation expectations:
- **Question 1**: We have heard of both qualitative and quantitative data from the previous consultant. What are the differences between the two? Should we use only one or both of these types of data and why? How can these be used in business predictions? Could you provide examples of each?
- **Question 2**: We have also heard a bit about the need for continuous improvement. Why should this be implemented, it seems like a waste of time. Why can’t we just implement the current project as it stands and move on to other pressing matters?
- **Question 3**: As a government, we adhere to all data protection requirements and have good governance in place. We only work with aggregated data and therefore will not expose any personal details. Have we covered everything from a data ethics standpoint? Is there anything else we need to implement from a data ethics perspective? 

### Question 1
- What is the difference between qualitative and quantitative data? How can these be used in business predictions?

Qualitative data is data is defined by categories and descriptions. The categories can have a distinct ordering, such as distinction levels for marks, but do not have to. The data values are assigned to categories based on the observation's characteristics. Participant's in a study on hair health may be classified by hair colour, for example. 

Quantitative data is numerical data that can be represented using an ordinal, interval or ratio scale. Observations can be discrete (meaning they can take on only fixed values in an interval) or continuous (meaning they can have take on any value along an interval). The weight of babies is a continuous, quantitative variable.

Quantitative data can be to forecast trends, determine the correlation between variables and predict future events. These statistical methods can be applied in business to better understand the relationships between customer behaviour and buying patterns, predict what effect changes in input prices will have on operations and plan for future stock. This is only a few of the ways quantitative data can be used to drive decision making. 

Qualitative data is used to provide insights for short-term decision making. This is can be in the form of an opinion poll on facebook to check which product clients would purchase, or measure customer sentiment towards an idea.

### Question 2
Continuous improvement ensures that your processes remain efficient and accurate. In using data from the web, continuously updating your information ensures the project produces the most accurate results. In the same vein, there are always faster and improved ways of coding and proccessing information being introduced. Keeping up with technology ensures the longevity of your project and will save time, money and effort in the long run. 

### Question 3